In [2]:
import pandas as pd
import json
from pathlib import Path

dependency_info_path = Path("../enriched_github_data/FOSS_projects_with_dependency_info.json")

# # Initialize an empty list to store the relevant data
# relevant_data = []

# # Open the large JSON file and process each entry
# with open(dependency_info_path, 'r', encoding='utf-8') as f:
#     # For each line in the file (assuming it's a JSON array of objects)
#     for line in f:
#         try:
#             entry = json.loads(line.strip())  # Load each JSON entry one by one
#             foss_project = entry.get("FOSS project")
#             dep_info = entry.get("dependency_info", {})

#             if foss_project and dep_info:
#                 dependencies = list(dep_info.keys())  # Adjust based on your need
#                 relevant_data.append({
#                     'FOSS project': foss_project,
#                     'dependencies': dependencies
#                 })
#         except json.JSONDecodeError as e:
#             print(f"Skipping invalid JSON entry: {e}")

# # Convert the relevant_data list into a pandas DataFrame
# df = pd.DataFrame(relevant_data)

# # Print or check the DataFrame
# print(df.head())

import ijson
import pandas as pd

# filename = "FOSS_projects_with_dependency_info.json"

# Open and stream the file
rows = []
with open(dependency_info_path, 'r', encoding='utf-8') as f:
    objects = ijson.items(f, 'item')  # 'item' targets elements in the top-level array
    for obj in objects:
        # Optionally filter here for memory efficiency
        rows.append({
            'FOSS project': obj.get('FOSS project'),
            'dependencies': list(obj.get('dependency_info', {}).keys())
        })

df = pd.DataFrame(rows)
print(df.head())



                          FOSS project  \
0  mtdvio/every-programmer-should-know   
1                storybookjs/storybook   
2                       facebook/react   
3     yangshun/tech-interview-handbook   
4                       996icu/996.ICU   

                                        dependencies  
0                                                 []  
1  [package.json, scripts/clean-merged-branches.s...  
2  [compiler/apps/playground/scripts/link-compile...  
3                     [package.json, pnpm-lock.yaml]  
4                                                 []  


In [3]:
print(df.tail())

                          FOSS project  \
13665  anuraghazra/github-readme-stats   
13666            fighting41love/funNLP   
13667            elastic/elasticsearch   
13668                      vitejs/vite   
13669             Snailclimb/JavaGuide   

                                            dependencies  
13665  [package-lock.json, package.json, scripts/push...  
13666                                                 []  
13667  [.buildkite/packer_cache.sh, .buildkite/script...  
13668  [package.json, pnpm-lock.yaml, scripts/docs-ch...  
13669                     [package.json, pnpm-lock.yaml]  


In [4]:
entry = df[df["FOSS project"] == "storybookjs/storybook"]
print(entry.to_dict(orient="records")[0])  # Pretty prints the first matching entry as a Python dict


{'FOSS project': 'storybookjs/storybook', 'dependencies': ['package.json', 'scripts/clean-merged-branches.sh', 'yarn.lock']}


In [5]:
print(df.columns)

Index(['FOSS project', 'dependencies'], dtype='object')


In [ ]:
import ijson
import pandas as pd
import json
rows = []
with open(dependency_info_path, 'r', encoding='utf-8') as f:
    objects = ijson.items(f, 'item')  # 'item' targets elements in the top-level array
    for obj in objects:
        project = obj.get('FOSS project')
        dep_info = obj.get('dependency_info', {})

        # Unescape each file's contents (they're JSON-escaped strings)
        unescaped_info = {}
        for filename, content in dep_info.items():
            try:
                unescaped_info[filename] = json.loads(content)
            except json.JSONDecodeError:
                # If it's not JSON (like .sh or .lock files), keep as raw string
                unescaped_info[filename] = content

        rows.append({
            'FOSS project': project,
            'dependency_info': unescaped_info,
            'dependency_filenames': list(dep_info.keys())
        })

df = pd.DataFrame(rows)


In [7]:
print(df.head())

                          FOSS project  \
0  mtdvio/every-programmer-should-know   
1                storybookjs/storybook   
2                       facebook/react   
3     yangshun/tech-interview-handbook   
4                       996icu/996.ICU   

                                     dependency_info  \
0                                                 {}   
1  {'package.json': {'name': '@storybook/root', '...   
2  {'compiler/apps/playground/scripts/link-compil...   
3  {'package.json': {'name': 'tech-interview-hand...   
4                                                 {}   

                                dependency_filenames  
0                                                 []  
1  [package.json, scripts/clean-merged-branches.s...  
2  [compiler/apps/playground/scripts/link-compile...  
3                     [package.json, pnpm-lock.yaml]  
4                                                 []  


In [10]:
entry = df[df["FOSS project"] == "storybookjs/storybook"]
print(entry.to_dict(orient="records")[0])  # Pretty prints the first matching entry as a Python dict

pkg_json = df[df['FOSS project'] == 'storybookjs/storybook'].iloc[0]['dependency_info']['package.json']
print(pkg_json['scripts'])  # prints the scripts object from package.json


{'FOSS project': 'storybookjs/storybook', 'dependency_info': {'package.json': {'name': '@storybook/root', 'scripts': {'ci-tests': 'cd code; yarn ci-tests', 'get-report-message': 'cd scripts; yarn get-report-message', 'get-template': 'cd scripts; yarn get-template', 'get-sandbox-dir': 'cd scripts; yarn get-sandbox-dir', 'i': 'yarn --cwd scripts && yarn --cwd code', 'knip': 'cd code; yarn knip', 'lint': 'cd code; yarn lint', 'nx': 'cd code; yarn nx', 'pretty-docs': 'cd scripts; yarn install >/dev/null; yarn docs:prettier:write', 'start': 'yarn task --task dev --template react-vite/default-ts --start-from=install', 'task': "echo 'Installing Script Dependencies...'; cd scripts; yarn install >/dev/null; cd ..; yarn --cwd=./scripts task", 'test': 'cd code; yarn test', 'upload-bench': 'cd scripts; yarn upload-bench', 'vite-ecosystem-ci:before-test': 'node ./scripts/vite-ecosystem-ci/before-test.js && cd ./sandbox/react-vite-default-ts && yarn install', 'vite-ecosystem-ci:build': 'yarn task --

In [11]:
# Convert DataFrame to JSON format
df.to_json("flattened_data.json", orient="records", lines=False,indent=2)


In [12]:
import json

def split_json_safely(input_file, output_prefix, max_size_mb=16):
    """
    Safely splits a large JSON array file into smaller files without cutting entries in half.
    :param input_file: Path to the large JSON file to be split.
    :param output_prefix: Prefix for the output split files.
    :param max_size_mb: Maximum size of each output file in MB (default is 16MB).
    """
    # Max size in bytes (default 16MB)
    max_size_bytes = max_size_mb * 1024 * 1024
    
    # Open the large file and start reading line by line
    with open(input_file, 'r', encoding='utf-8') as f:
        chunk = []
        chunk_size = 0
        chunk_count = 0
        current_json = ''
        
        for line in f:
            current_json += line.strip()  # Accumulate the line into the current JSON object
            
            try:
                # Try parsing the accumulated string as JSON
                doc = json.loads(current_json)
                # If parsing succeeds, we have a complete document
                
                chunk.append(doc)
                chunk_size += len(json.dumps(doc))  # Calculate the approximate size of this document

                # If the chunk size exceeds the limit, write to a new file
                if chunk_size > max_size_bytes:
                    chunk_count += 1
                    output_file = f"{output_prefix}_{chunk_count}.json"
                    with open(output_file, 'w', encoding='utf-8') as out_f:
                        json.dump(chunk, out_f)
                    
                    # Reset for the next chunk
                    chunk = [doc]
                    chunk_size = len(json.dumps(doc))
                current_json = ''  # Reset after successfully parsing a complete document
            except json.JSONDecodeError:
                # If the current string is not a complete JSON object, keep accumulating lines
                continue
        
        # Write any remaining documents in the last chunk
        if chunk:
            chunk_count += 1
            output_file = f"{output_prefix}_{chunk_count}.json"
            with open(output_file, 'w', encoding='utf-8') as out_f:
                json.dump(chunk, out_f)
        
    print(f"Split completed. {chunk_count} files created.")

# Call the function to split the large JSON file
split_json_safely('flattened_data.json', 'split_data')


KeyboardInterrupt: 